## Set Up ##

In [1]:
import os
import shutil
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time
import openpyxl
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
import pandas as pd
import xlwings as xw
import numpy as np
start_master = time.time()

In [2]:
#download staffing, exec pivot, the 3 TBM ESC files (https://app.powerbigov.us/groups/711b4b6c-92eb-4c57-badc-11b293e42bf1/list)
#, last month's folder, cloud file (if possible)

In [3]:
download_path = input("Enter path to folder where all raw sources files are downloaded: ")
#C:\Users\stephalin\Downloads
raw_Staff = os.path.join(download_path, input("Enter Staffing source file name: ") + ".xlsx")
raw_EP = os.path.join(download_path, input("Enter Execution Pivot source file name: ") + ".xlsx")
src_folder = os.path.join(download_path, input("Enter source folder name: "))
raw_Cloud = os.path.join(download_path, input("Enter ESC Cloud source file name: ") + ".csv")

raw_CostPool = os.path.join(download_path,"TBM ESC Cost Pool.xlsx")
raw_Sol = os.path.join(download_path,"TBM ESC Solutions.xlsx")
raw_T = os.path.join(download_path,"TBM ESC Tower.xlsx")


Enter path to folder where all raw sources files are downloaded:  C:\Users\stephalin\Downloads
Enter Staffing source file name:  RMS Staffing Projections & Actuals_20250908
Enter Execution Pivot source file name:  RMS Execution Pivot Report_20250908
Enter source folder name:  FY25 July
Enter ESC Cloud source file name:  OCT-AUG ALL ESCID REPORT


### Copying and renaming source folder ###

In [4]:
today = datetime.today()

#to test on last month's data, uncomment
#today = today - relativedelta(months=1)

print("Today is:", today)
first_of_month = today.replace(day=1)
prior_month_date = first_of_month - timedelta(days=1)
print("We will create data inputs for", prior_month_date.strftime("%B"), prior_month_date.year, "dashboard")

Today is: 2025-09-12 15:11:32.833285
We will create data inputs for August 2025 dashboard


In [5]:
#src_folder = r'C:\Users\stephalin\Downloads\FY25 June - Q3'
#dst_folder = r'C:\Users\stephalin\Downloads\FY25 July'
first_of_month = today.replace(day=1)
prior_month_date = first_of_month - timedelta(days=1)
year_str = str(prior_month_date.year)[-2:]
month_str = prior_month_date.strftime("%B")
quarter_map = {3: "Q1", 6: "Q2", 9: "Q3", 12: "Q4"}
quarter_suffix = ""
if prior_month_date.month in quarter_map:
    quarter_suffix = f" - {quarter_map[prior_month_date.month]}"

parent_folder = os.path.dirname(src_folder)
dst_folder_name = f"FY{year_str} {month_str}{quarter_suffix}"
dst_folder = os.path.join(parent_folder, dst_folder_name)

date_str = datetime.now().strftime('%Y%m%d')

os.makedirs(dst_folder, exist_ok=True)
for root, dirs, files in os.walk(src_folder):
    for filename in files:
        ext = os.path.splitext(filename)[1].lower()
        src_path = os.path.join(root, filename)
        if ext == ".xlsm":
            new_name = f"Model Enhancements_{date_str}{ext}"
        elif "Golden PowerBI Source File" in filename and ext == ".xlsx":
            new_name = f"Golden PowerBI Source File_{date_str}{ext}"
        elif ext == ".pbix":
            new_name = f"FY2025 OCIO TBM Analytics{ext}"
        else:
            continue
        shutil.copy2(src_path, os.path.join(dst_folder, new_name))

print("Copying contents of " + src_folder + " into " + dst_folder)
raw_data_src = os.path.join(src_folder, "Raw Data")
raw_data_dst = os.path.join(dst_folder, "Raw Data")
if os.path.exists(raw_data_src):
    shutil.copytree(raw_data_src, raw_data_dst, dirs_exist_ok=True)

for filename in os.listdir(raw_data_dst):
    file_path = os.path.join(raw_data_dst, filename)
    if os.path.isfile(file_path):
        base, ext = os.path.splitext(filename)
        base = base.split("_")[0]
        new_name = f"{base}_{date_str}{ext}"
        new_path = os.path.join(raw_data_dst, new_name)
        os.rename(file_path, new_path)
print("All contents have been copied and renamed in "+ dst_folder)


Copying contents of C:\Users\stephalin\Downloads\FY25 July into C:\Users\stephalin\Downloads\FY25 August
All contents have been copied and renamed in C:\Users\stephalin\Downloads\FY25 August


## Refreshing data inputs ##

### Execution Pivot ###

In [6]:
start1 = time.time()
dst_EP = raw_data_dst+"\RMS Execution Pivot Report_"+date_str+".xlsx"
print("Copying contents of "+raw_EP+" into "+dst_EP)
print(" ")

Copying contents of C:\Users\stephalin\Downloads\RMS Execution Pivot Report_20250908.xlsx into C:\Users\stephalin\Downloads\FY25 August\Raw Data\RMS Execution Pivot Report_20250912.xlsx
 


<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_21336\1570110182.py:2: SyntaxWarning: invalid escape sequence '\R'
  dst_EP = raw_data_dst+"\RMS Execution Pivot Report_"+date_str+".xlsx"


In [7]:
# Read raw data
raw_df = pd.read_excel(raw_EP, sheet_name="Sheet1")

# Cleaned data
df = raw_df.rename(columns={
    "UFMS Vendor Recipient": "Vendor Recipient",
    "Vendor Name": "Vendor POC Name",
    "Vendor Email": "Vendor POC Email"
})
delete_cols = [
    "Projection Note",
    "Line Item Mechanism Target Award Date",
    "RAL",
    "Late Waiver Number",
    "Late Waiver Deadline",
    "Unreconciled Commitments",
    "Unreconciled Obligations"
]
df = df.drop(columns=delete_cols, errors="ignore")

with xw.App(visible=False) as app:
    wb = app.books.open(dst_EP)
    # Write cleaned data to "Sheet1"
    try:
        wb.sheets["Sheet1"].delete()
    except Exception:
        pass
    clean_sht = wb.sheets.add("Sheet1", before=wb.sheets[0])
    clean_sht.range("A1").options(index=False).value = df

    nrows, ncols = df.shape
    # +1 for header row
    table_range = clean_sht.range((1, 1), (nrows + 1, ncols)).api
    clean_table = clean_sht.api.ListObjects.Add(1, table_range, None, 1)
    clean_table.Name = "Table13"
    clean_table.TableStyle = "TableStyleMedium2"

    wb.save()
    wb.close()

EP_len = len(df)

In [8]:
end1 = time.time()
print(f"Elapsed time: {end1 - start1} seconds for EP")

Elapsed time: 30.030613899230957 seconds for EP


### Staffing Report ###

In [9]:
start2 = time.time()
dst_Staff = raw_data_dst+"\RMS Staffing Projections & Actuals_"+date_str+".xlsx"
print("Copying contents of "+raw_Staff+" into "+dst_Staff)

Copying contents of C:\Users\stephalin\Downloads\RMS Staffing Projections & Actuals_20250908.xlsx into C:\Users\stephalin\Downloads\FY25 August\Raw Data\RMS Staffing Projections & Actuals_20250912.xlsx


<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_21336\4235163783.py:2: SyntaxWarning: invalid escape sequence '\R'
  dst_Staff = raw_data_dst+"\RMS Staffing Projections & Actuals_"+date_str+".xlsx"


In [10]:
#Staffing report

df = pd.read_excel(raw_Staff, sheet_name="Sheet1")
wcf_map = pd.read_excel(dst_Staff, sheet_name="WCF Mapping")
ocio_map = pd.read_excel(dst_Staff, sheet_name="OCIO Admin Codes")
Staff_len = len(df)

# Remove 'AFPS' from column headers
df.columns = [col.replace("AFPS", "").strip() for col in df.columns]

# Insert Full Name column
def full_name_formula(row):
    if pd.notnull(row.iloc[0]) and "Budget Staff Adjustment" in str(row.iloc[0]):
        return ""
    if (str(row.get("Last Name", "")) + str(row.get("First Name", ""))).strip() == "":
        return "Vacant"
    return f"{row.get('Last Name','')}, {row.get('First Name','')}".strip(", ")
record_id_idx = df.columns.get_loc("Record ID")
df.insert(record_id_idx, "Full Name", df.apply(full_name_formula, axis=1))

# Find the index after "Total Amount"
total_amt_idx = df.columns.get_loc("Total Amount") + 1

# Insert in reverse order
df.insert(total_amt_idx, "OCIO Assigned Office", "")
df.insert(total_amt_idx, "OCIO Funding Office", "")
df.insert(total_amt_idx, "Service Line", "")

In [11]:
df['Service Line'] = df['Payroll Dist. WCF Indicator'].map(
    wcf_map.set_index('WCFID')['FY21-22 Service Line Descriptions']
)
#df

In [12]:
df['EAC'] = np.where(
    df['Employee Admin Code'].str.strip() == '',
    '',
    'H' + df['Employee Admin Code'].str.split(' ').str[0]
)
#df[['Employee Admin Code','EAC']]

In [13]:
df['OCIO Funding Office']= df['EAC'].map(ocio_map.set_index('Admin Code')['Office Abrv']).fillna('')
df['OCIO Assigned Office']= df['EAC'].map(ocio_map.set_index('Admin Code')['Office Abrv']).fillna('')

In [14]:
with xw.App(visible=False) as app:
    app.display_alerts = False
    app.screen_updating = False
    wb = app.books.open(dst_Staff)
    try:
        wb.sheets["Import Data"].delete()
    except Exception:
        pass
    sht = wb.sheets.add("Import Data", before=wb.sheets[0])
    sht.range("A1").options(index=False).value = df

    nrows, ncols = df.shape
    table_range = sht.range((1, 1), (nrows + 1, ncols)).api
    table = sht.api.ListObjects.Add(1, table_range, None, 1)
    table.Name = "Table13"

    wb.save()
    wb.close()

In [15]:
Staff_len = len(df)

In [16]:
end2 = time.time()
print(f"Elapsed time: {end2 - start2} seconds for Staffing")

Elapsed time: 131.1748673915863 seconds for Staffing


### TBM ESC ###

In [17]:
start3 = time.time()
dst_CostPool = raw_data_dst+"\TBM ESC Cost Pool_"+date_str+".xlsx"
print("Copying contents of "+raw_CostPool+" into "+dst_CostPool)

Copying contents of C:\Users\stephalin\Downloads\TBM ESC Cost Pool.xlsx into C:\Users\stephalin\Downloads\FY25 August\Raw Data\TBM ESC Cost Pool_20250912.xlsx


<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_21336\2298528951.py:2: SyntaxWarning: invalid escape sequence '\T'
  dst_CostPool = raw_data_dst+"\TBM ESC Cost Pool_"+date_str+".xlsx"


In [ ]:
cp_df = pd.read_excel(raw_CostPool, sheet_name="TBM ESC Cost Pool", header=1)
#df = df.iloc[1:].reset_index(drop=True)
CostPoolRaw_len = len(cp_df)

# Find the index after "Total Amount"
total_amt_idx = cp_df.columns.get_loc("CP Amount") + 1

# Insert in reverse order
cp_df.insert(total_amt_idx, "Service Line", "")
cp_df.insert(total_amt_idx, "Office", "")
cp_df.insert(total_amt_idx, "In OCIO?", "")

with pd.ExcelWriter(dst_CostPool, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    cp_df.to_excel(writer, sheet_name="TBM ESC Cost Pool", index=False)

wb = openpyxl.load_workbook(dst_CostPool)
ws = wb["TBM ESC Cost Pool"]


In [ ]:
header = [cell.value for cell in ws[1]]
admin_code_col = header.index("Admin Code") + 1
investment_id_col = header.index("Investment ID") + 1
in_ocio_col = header.index("In OCIO?") + 1
office_col = header.index("Office") + 1
service_line_col = header.index("Service Line") + 1

for row in range(2, ws.max_row + 1):
    p_cell = f"{openpyxl.utils.get_column_letter(admin_code_col)}{row}"
    j_cell = f"{openpyxl.utils.get_column_letter(investment_id_col)}{row}"
    ws.cell(row=row, column=in_ocio_col).value = f'=ISNUMBER(SEARCH("HCAJR",{p_cell}))'
    ws.cell(row=row, column=office_col).value = (f'=IF(ISNA(VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE)),"",VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE))')    
    ws.cell(row=row, column=service_line_col).value = (f'=IF(ISNA(VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE)),"Other - Non OCIO",VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE))')   

wb.save(dst_CostPool)

In [20]:
dst_Sol = raw_data_dst+"\TBM ESC Solutions_"+date_str+".xlsx"
print("Copying contents of "+raw_Sol+" into "+dst_Sol)

df = pd.read_excel(raw_Sol, sheet_name="TBM ESC Solutions", header=1)
#df = df.iloc[1:].reset_index(drop=True)
SolRaw_len = len(df)


# Find the index after "Total Amount"
total_amt_idx = df.columns.get_loc("Amount") + 1

# Insert in reverse order
df.insert(total_amt_idx, "Service Line", "")
df.insert(total_amt_idx, "Office", "")
df.insert(total_amt_idx, "In OCIO?", "")

with pd.ExcelWriter(dst_Sol, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="TBM ESC Solutions", index=False)

wb = openpyxl.load_workbook(dst_Sol)
ws = wb["TBM ESC Solutions"]


header = [cell.value for cell in ws[1]]
admin_code_col = header.index("Admin Code") + 1
investment_id_col = header.index("Investment ID") + 1
in_ocio_col = header.index("In OCIO?") + 1
office_col = header.index("Office") + 1
service_line_col = header.index("Service Line") + 1

for row in range(2, ws.max_row + 1):
    p_cell = f"{openpyxl.utils.get_column_letter(admin_code_col)}{row}"
    j_cell = f"{openpyxl.utils.get_column_letter(investment_id_col)}{row}"
    ws.cell(row=row, column=in_ocio_col).value = f'=ISNUMBER(SEARCH("HCAJR",{p_cell}))'
    ws.cell(row=row, column=office_col).value = (f'=IF(ISNA(VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE)),"",VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE))')    
    ws.cell(row=row, column=service_line_col).value = (f'=IF(ISNA(VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE)),"Other - Non OCIO",VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE))')   

wb.save(dst_Sol)

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_21336\688287672.py:1: SyntaxWarning: invalid escape sequence '\T'
  dst_Sol = raw_data_dst+"\TBM ESC Solutions_"+date_str+".xlsx"


Copying contents of C:\Users\stephalin\Downloads\TBM ESC Solutions.xlsx into C:\Users\stephalin\Downloads\FY25 August\Raw Data\TBM ESC Solutions_20250912.xlsx


In [21]:
dst_T = raw_data_dst+"\TBM ESC Tower_"+date_str+".xlsx"
print("Copying contents of "+raw_T+" into "+dst_T)

df = pd.read_excel(raw_T, sheet_name="TBM ESC Tower", header=1)
#df = df.iloc[1:].reset_index(drop=True)
TowerRaw_len = len(df)

# Find the index after "Total Amount"
total_amt_idx = df.columns.get_loc("Amount") + 0

# Insert in reverse order
df.insert(total_amt_idx, "Service Line", "")
df.insert(total_amt_idx, "Office", "")
df.insert(total_amt_idx, "In OCIO?", "")

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_21336\299851042.py:1: SyntaxWarning: invalid escape sequence '\T'
  dst_T = raw_data_dst+"\TBM ESC Tower_"+date_str+".xlsx"


Copying contents of C:\Users\stephalin\Downloads\TBM ESC Tower.xlsx into C:\Users\stephalin\Downloads\FY25 August\Raw Data\TBM ESC Tower_20250912.xlsx


In [22]:
with pd.ExcelWriter(dst_T, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="TBM ESC Tower", index=False)

wb = openpyxl.load_workbook(dst_T)
ws = wb["TBM ESC Tower"]

header = [cell.value for cell in ws[1]]
admin_code_col = header.index("Admin Code") + 1
investment_id_col = header.index("Investment ID") + 1
in_ocio_col = header.index("In OCIO?") + 1
office_col = header.index("Office") + 1
service_line_col = header.index("Service Line") + 1

for row in range(2, ws.max_row + 1):
    p_cell = f"{openpyxl.utils.get_column_letter(admin_code_col)}{row}"
    j_cell = f"{openpyxl.utils.get_column_letter(investment_id_col)}{row}"
    ws.cell(row=row, column=in_ocio_col).value = f'=ISNUMBER(SEARCH("HCAJR",{p_cell}))'
    ws.cell(row=row, column=office_col).value = (f'=IF(ISNA(VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE)),"",VLOOKUP({p_cell},\'OCIO ADMIN CODES\'!A:I,9,FALSE))')    
    ws.cell(row=row, column=service_line_col).value = (f'=IF(ISNA(VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE)),"Other - Non OCIO",VLOOKUP({j_cell},\'FY25 Service Lines\'!A:C,3,FALSE))')   

wb.save(dst_T)

In [23]:
print("Row counts for raw data. Check with refreshed data.")
print(f"{EP_len} rows for Execution Pivot")
print(f"{Staff_len} rows for Staffing")
print(f"{CostPoolRaw_len} rows for Cost Pool")
print(f"{SolRaw_len} rows for Solution")
print(f"{TowerRaw_len} rows for Tower")

Row counts for raw data. Check with refreshed data.
13794 rows for Execution Pivot
10248 rows for Staffing
18912 rows for Cost Pool
14184 rows for Solution
26004 rows for Tower


In [24]:
end3 = time.time()
print(f"Elapsed time: {(end3 - start3)/60} minutes for TBM ESC files")

Elapsed time: 4.848472837607066 minutes for TBM ESC files


In [25]:
print(f"Elapsed time: {(end3 - start_master)/60} minutes for process up to now")

Elapsed time: 7.923545789718628 minutes for process up to now


## Refreshing Cloud Data ##

In [36]:
#build TBM Tower ratios (pivot, filter for FY25)
tower_2025 = df[(df['Fiscal Year'] == 2025) & (df['Amount'] != 0)]
tower_pivot = tower_2025.pivot_table(
    index=["ESC ID", "Towers"],
    values=["Amount"],
    aggfunc="sum"
).reset_index()

tower_pivot['% of ESC ID'] = (
    pivot.groupby('ESC ID')['Amount']
    .transform(lambda x: x / x.sum())
)

tower_pivot

,ESC ID,Towers,Amount,% of ESC ID
0,4,Delivery,85534,0.914920
1,4,IT Management,4533,0.048488
2,4,Security & Compliance,3421,0.036593
3,6,Application,12174,0.660016
4,6,Delivery,2213,0.119978
...,...,...,...,...
2370,3519,Application,35250,0.128626
2371,3519,Platform,124800,0.455391
2372,3519,Security & Compliance,114000,0.415982
2373,3520,Delivery,42500,0.944444


In [ ]:
#build TBM CP ratios (pivot, filter for FY25)

In [ ]:
#pivot export on cloud data (actuals)

In [ ]:
#create tower breakdown (join actuals and ratios)
#create CP breakdown (join actuals and ratios)

In [ ]:
input("STOP! \n 1. Check if the row counts match. \n 2. Refresh the cloud data & pivots on EP data \n 3. Run the Model Enhancements macro \n Then press Enter to continue")

## Refreshing Golden Source ##

In [3]:
import os
import shutil
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time
import openpyxl
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
import pandas as pd
import xlwings as xw
import numpy as np
start_gc = time.time()

download_path = input("Enter path to folder where everything is downloaded: ")
src_folder = os.path.join(download_path, input("Enter source folder name: "))

#C:\Users\stephalin\Downloads

today = datetime.today()
first_of_month = today.replace(day=1)
prior_month_date = first_of_month - timedelta(days=1)
year_str = str(prior_month_date.year)[-2:]
month_str = prior_month_date.strftime("%B")
quarter_map = {3: "Q1", 6: "Q2", 9: "Q3", 12: "Q4"}
quarter_suffix = ""
if prior_month_date.month in quarter_map:
    quarter_suffix = f" - {quarter_map[prior_month_date.month]}"

parent_folder = os.path.dirname(src_folder)
dst_folder_name = f"FY{year_str} {month_str}{quarter_suffix}"
dst_folder = os.path.join(parent_folder, dst_folder_name)

date_str = input("Enter date string of golden source + TBM model [YYYYMMDD]")

Enter path to folder where everything is downloaded:  C:\Users\stephalin\Downloads
Enter source folder name:  FY25 July
Enter date string of golden source + TBM model [YYYYMMDD] 20250908


In [4]:
g_src = dst_folder+"\Golden PowerBI Source File_"+date_str+".xlsx"
#g_src = dst_folder+"\Golden PowerBI Source File_"+date_str+".xlsx"

gs_df = pd.read_excel(g_src, sheet_name="Golden - Spend Tracking", header=0)
#gs_df

model = dst_folder+"\Model Enhancements_"+date_str+".xlsm"

mod_df = pd.read_excel(model, sheet_name="Combined", header=0)
#mod_df

<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:7: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:7: SyntaxWarning: invalid escape sequence '\M'
C:\Users\stephalin\AppData\Local\Temp\ipykernel_15604\2708958362.py:1: SyntaxWarning: invalid escape sequence '\G'
  g_src = dst_folder+"\Golden PowerBI Source File_"+date_str+".xlsx"
C:\Users\stephalin\AppData\Local\Temp\ipykernel_15604\2708958362.py:7: SyntaxWarning: invalid escape sequence '\M'
  model = dst_folder+"\Model Enhancements_"+date_str+".xlsm"


In [5]:
#pivot combined tab in TBM Model file by Org Symbol,  Oblig, Expenditure
#filter out CURRENT month, not month of dashboard
month_to_filter = today.strftime("%b").upper() + today.strftime("-%y")
filtered = mod_df[mod_df["Accounting Period"] != month_to_filter]
pivot = filtered.pivot_table(
    index="Org Symbol",
    values=["Obligations", "Expenditures", "Ceiling"],
    aggfunc="sum"
)
pivot

,Ceiling,Expenditures,Obligations
Org Symbol,,,
CEO,53306760.0,25520489.26,4.191762e+07
CSPO,48375020.0,18877682.81,4.131974e+07
DSO,205536508.0,66616513.92,1.649439e+08
OD,72479335.0,32275953.65,5.902088e+07


In [6]:
#throw them into cum oblig , cum exp on golden source file
this_year = prior_month_date.year
fy_str = f"FY{str(this_year)[-2:]}"  # "FY25"
monthtofill = (gs_df["FY"] == fy_str) & (gs_df["Month"] == month_str)

# Ensure the lengths of mask and pivot values match
mask = (gs_df["FY"] == fy_str) & (gs_df["Month"] == month_str)
rows_to_update = gs_df.loc[mask]

# Check if lengths match before assignment
if len(rows_to_update) == len(pivot):
    gs_df.loc[mask, "Cumulative Obl"] = pivot["Obligations"].values
    gs_df.loc[mask, "Cumulative Exp"] = pivot["Expenditures"].values
    gs_df.loc[mask, "Budget Ceiling"] = pivot["Ceiling"].values
else:
    print("Row count mismatch: cannot assign values safely.")

gs_df[mask]

,Office,Month,Month Order,CPIC,Budget Ceiling,Obligations,Expenditures,Cumulative Obl,Cumulative Exp,Obligation Projection,Org Symbol,FY
232,CUSTOMER ENGAGEMENT OFFICE,August,11,53350313,53306760.0,NaN,NaN,4.191762e+07,25520489.26,4.071219e+07,CEO,FY25
233,CYBERSECURITY PROGRAM OFFICE,August,11,45536505,48375020.0,NaN,NaN,4.131974e+07,18877682.81,4.438794e+07,CSPO,FY25
234,DIGITAL SERVICES OFFICE,August,11,215002493,205536508.0,NaN,NaN,1.649439e+08,66616513.92,1.382216e+08,DSO,FY25
235,OFFICE OF THE DIRECTOR,August,11,51740257,72479335.0,NaN,NaN,5.902088e+07,32275953.65,5.925131e+07,OD,FY25


In [7]:
#Calculate Obligations and Expenditures (this month's Cum Obl or Exp - last month's)
months_order = ["October", "November", "December", "January", "February", "March", "April", "May", "June","July", "August", "September"]
gs_df["Month"] = pd.Categorical(gs_df["Month"], categories=months_order, ordered=True)

prev_mask = (gs_df["FY"] == fy_str) & (gs_df["Month"] == gs_df["Month"].cat.categories[gs_df["Month"].cat.categories.get_loc(month_str)-1])

gs_df.loc[mask, "Obligations"] = gs_df.loc[mask, "Cumulative Obl"].values - gs_df.loc[prev_mask, "Cumulative Obl"].values
gs_df.loc[mask, "Expenditures"] = gs_df.loc[mask, "Cumulative Exp"].values - gs_df.loc[prev_mask, "Cumulative Exp"].values

#gs_df[mask]

In [8]:
#ensure budget ceiling #s are consistent for remainder of year

months_order = ["October", "November", "December", "January", "February", "March", "April", "May", "June","July", "August", "September"]
gs_df["Month"] = pd.Categorical(gs_df["Month"], categories=months_order, ordered=True)

source_mask = (gs_df["Month"] == month_str) & (gs_df["FY"] == fy_str)
values = gs_df.loc[source_mask, "Budget Ceiling"].values

after_mask = (gs_df["Month"] >= month_str) & (gs_df["FY"] == fy_str)
n = after_mask.sum()
gs_df.loc[after_mask, "Budget Ceiling"] = np.tile(values, int(np.ceil(n/len(values))))[:n]
gs_df[after_mask]

,Office,Month,Month Order,CPIC,Budget Ceiling,Obligations,Expenditures,Cumulative Obl,Cumulative Exp,Obligation Projection,Org Symbol,FY
232,CUSTOMER ENGAGEMENT OFFICE,August,11,53350313,53306760.0,9038957.83,2278491.66,4.191762e+07,25520489.26,4.071219e+07,CEO,FY25
233,CYBERSECURITY PROGRAM OFFICE,August,11,45536505,48375020.0,1252558.86,2025255.31,4.131974e+07,18877682.81,4.438794e+07,CSPO,FY25
234,DIGITAL SERVICES OFFICE,August,11,215002493,205536508.0,66442672.11,4716408.65,1.649439e+08,66616513.92,1.382216e+08,DSO,FY25
235,OFFICE OF THE DIRECTOR,August,11,51740257,72479335.0,4896518.52,1787461.95,5.902088e+07,32275953.65,5.925131e+07,OD,FY25
236,CUSTOMER ENGAGEMENT OFFICE,September,12,53350313,53306760.0,NaN,NaN,NaN,NaN,4.854572e+07,CEO,FY25
237,CYBERSECURITY PROGRAM OFFICE,September,12,45536505,48375020.0,NaN,NaN,NaN,NaN,4.870870e+07,CSPO,FY25
238,DIGITAL SERVICES OFFICE,September,12,215002493,205536508.0,NaN,NaN,NaN,NaN,1.779420e+08,DSO,FY25
239,OFFICE OF THE DIRECTOR,September,12,51740257,72479335.0,NaN,NaN,NaN,NaN,6.437825e+07,OD,FY25


In [ ]:
#replace cpic proj (golden source, CPIC col), update for the month + rest of year (use esc tower data pivot: fiscal year 2025, In OCIO = True)

In [9]:
GL_df = pd.read_excel(model, sheet_name="GL + Lookups", header=0)
#GL_df

In [10]:
# Fiscal quarters: 3 months each
quarters = [months_order[i:i+3] for i in range(0, 12, 3)]

#month_str = 'October'
month_idx = months_order.index(month_str)

# Find current quarter
current_qtr_idx = month_idx // 3

#Calculate Upcoming quarters (excluding current)
upcoming_quarters = len(quarters) - (current_qtr_idx + 1)
upcoming_quarters

0

In [11]:
#pivot by OCIO office, total proj oblig (only for Q4)
#For Q1-3, add UFMS commitments [QTR X] + Manual commitments [QTR X] + Other Projections [QTR X] + Payroll Proj [QTR X] + UFMS Obligations [QTR X] + UFMS Oblig
#Do this cumulatively

quarters = [
    ["UFMS Commitments Qtr 1", "Manual Commitments Qtr 1", "Other Projections Qtr 1", "Payroll Projections Qtr 1", "UFMS Obligations Qtr 1", "UFMS Obligations"],
    ["UFMS Commitments Qtr 2", "Manual Commitments Qtr 2", "Other Projections Qtr 2", "Payroll Projections Qtr 2", "UFMS Obligations Qtr 2", "UFMS Obligations"],
    ["UFMS Commitments Qtr 3", "Manual Commitments Qtr 3", "Other Projections Qtr 3", "Payroll Projections Qtr 3", "UFMS Obligations Qtr 3", "UFMS Obligations"]
]

# Cumulative sums for Qtr 1, Qtr 2, Qtr 3
for i in range(3):
    cols = [col for q in quarters[:i+1] for col in q if col in GL_df.columns]
    GL_df[f"Qtr_{i+1}"] = GL_df[cols].sum(axis=1)

# Qtr 4 is just "Total Projected Obligations"
GL_df["Qtr_4"] = GL_df["Total Projected Obligations"]

Val = [f"Qtr_{i+1}" for i in range(4)]

pivot_GL = GL_df.pivot_table(
    index="OCIO Office",
    values=Val,
    aggfunc="sum"
)

pivot_GL


,Qtr_1,Qtr_2,Qtr_3,Qtr_4
OCIO Office,,,,
CEO,5.321688e+07,1.052263e+08,1.731877e+08,4.390343e+07
CSPO,4.552546e+07,1.004238e+08,1.623233e+08,4.813040e+07
DSO,1.748039e+08,3.969006e+08,5.968766e+08,1.738227e+08
OD,6.295224e+07,1.637015e+08,2.308899e+08,6.293546e+07


In [12]:
#delete oblig projections for clean slate
gs_df['Obligation Projection'] = np.nan

#fill in Oblig Projection based on GL Lookup pivot #s. quarter months still upcoming will be imputed
if upcoming_quarters > 0:
    quarter_months = {1: "December", 2: "March", 3: "June", 4: "September"}
    for q in range(1, upcoming_quarters + 1):
        month = quarter_months[q]
        mask = (gs_df["Month"] == month) & (gs_df["FY"] == fy_str)
        gs_df.loc[mask, "Obligation Projection"] = gs_df.loc[mask, "Org Symbol"].map(pivot_GL[f"Qtr_{q}"])
elif upcoming_quarters == 0:
    mask = (gs_df["Month"] == "September") & (gs_df["FY"] == fy_str)
    gs_df.loc[mask, "Obligation Projection"] = gs_df.loc[mask, "Org Symbol"].map(pivot_GL["Qtr_4"])


#gs_df[gs_df.Month == month_str]

In [13]:

#Cum Obl for this month is Obl Prjection
mask = (gs_df["Month"] == month_str) & (gs_df["FY"] == fy_str)
gs_df.loc[mask, "Obligation Projection"] = gs_df.loc[mask, "Cumulative Obl"]

#gs_df[after_mask]
gs_df[gs_df.Month == month_str]

,Office,Month,Month Order,CPIC,Budget Ceiling,Obligations,Expenditures,Cumulative Obl,Cumulative Exp,Obligation Projection,Org Symbol,FY
40,CUSTOMER ENGAGEMENT OFFICE,August,11,31566200,3.130020e+07,4957998.93,1035122.54,3.013076e+07,15634075.81,NaN,CEO,FY21
41,CYBERSECURITY PROGRAM OFFICE,August,11,40850112,4.229011e+07,762511.50,770330.50,4.089137e+07,24285280.76,NaN,CSPO,FY21
42,DIGITAL SERVICES OFFICE,August,11,133495328,1.378394e+08,3212179.86,1706680.25,1.297704e+08,61414904.72,NaN,DSO,FY21
43,OFFICE OF THE DIRECTOR,August,11,61875146,7.179911e+07,759576.37,759407.92,6.189129e+07,30121673.25,NaN,OD,FY21
88,CUSTOMER ENGAGEMENT OFFICE,August,11,40670780,4.471200e+07,5943413.43,1094976.42,4.216647e+07,17196889.05,NaN,CEO,FY22
89,CYBERSECURITY PROGRAM OFFICE,August,11,42750654,4.872483e+07,1053905.21,891179.46,4.758620e+07,26704479.37,NaN,CSPO,FY22
90,DIGITAL SERVICES OFFICE,August,11,156500825,1.525906e+08,3427160.58,1840305.00,1.469087e+08,60376567.56,NaN,DSO,FY22
91,OFFICE OF THE DIRECTOR,August,11,206252401,2.471257e+08,935073.39,807961.50,2.400435e+08,53164631.69,NaN,OD,FY22
136,CUSTOMER ENGAGEMENT OFFICE,August,11,43432223,5.025915e+07,7817894.74,1267876.71,4.857713e+07,12896086.46,NaN,CEO,FY23
137,CYBERSECURITY PROGRAM OFFICE,August,11,43927851,4.772483e+07,1030707.93,1034557.55,4.662109e+07,16559626.77,NaN,CSPO,FY23


In [14]:
# Sort the DataFrame
gs_df.sort_values(by=["FY", "Office", "Month Order"], inplace=True)

# Interpolate missing values in 'Obligation Projection'
gs_df['Obligation Projection'] = gs_df.groupby(['FY', 'Office'])['Obligation Projection'].transform(lambda group: group.interpolate())
#re-sort
gs_df.sort_values(by=["FY", "Month Order","Office"], inplace=True)

#gs_df[after_mask]
gs_df[gs_df.Month == 'August']

,Office,Month,Month Order,CPIC,Budget Ceiling,Obligations,Expenditures,Cumulative Obl,Cumulative Exp,Obligation Projection,Org Symbol,FY
40,CUSTOMER ENGAGEMENT OFFICE,August,11,31566200,3.130020e+07,4957998.93,1035122.54,3.013076e+07,15634075.81,NaN,CEO,FY21
41,CYBERSECURITY PROGRAM OFFICE,August,11,40850112,4.229011e+07,762511.50,770330.50,4.089137e+07,24285280.76,NaN,CSPO,FY21
42,DIGITAL SERVICES OFFICE,August,11,133495328,1.378394e+08,3212179.86,1706680.25,1.297704e+08,61414904.72,NaN,DSO,FY21
43,OFFICE OF THE DIRECTOR,August,11,61875146,7.179911e+07,759576.37,759407.92,6.189129e+07,30121673.25,NaN,OD,FY21
88,CUSTOMER ENGAGEMENT OFFICE,August,11,40670780,4.471200e+07,5943413.43,1094976.42,4.216647e+07,17196889.05,NaN,CEO,FY22
89,CYBERSECURITY PROGRAM OFFICE,August,11,42750654,4.872483e+07,1053905.21,891179.46,4.758620e+07,26704479.37,NaN,CSPO,FY22
90,DIGITAL SERVICES OFFICE,August,11,156500825,1.525906e+08,3427160.58,1840305.00,1.469087e+08,60376567.56,NaN,DSO,FY22
91,OFFICE OF THE DIRECTOR,August,11,206252401,2.471257e+08,935073.39,807961.50,2.400435e+08,53164631.69,NaN,OD,FY22
136,CUSTOMER ENGAGEMENT OFFICE,August,11,43432223,5.025915e+07,7817894.74,1267876.71,4.857713e+07,12896086.46,NaN,CEO,FY23
137,CYBERSECURITY PROGRAM OFFICE,August,11,43927851,4.772483e+07,1030707.93,1034557.55,4.662109e+07,16559626.77,NaN,CSPO,FY23


In [15]:
with pd.ExcelWriter(g_src, mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
    gs_df.to_excel(writer, sheet_name='Golden - Spend Tracking', index=False)

In [19]:
end = time.time()

print(f"Elapsed time: {(end - start_gc)} seconds for GC refresh")

Elapsed time: 112.68083238601685 seconds for GC refresh


In [17]:
print(f"Elapsed time: {(end - start_master)/60} minutes TOTAL")

NameError: name 'end' is not defined